# To send personalized table as attachment to ADM vendors

### SOP for using this notebook to send personalized table from a dump file as attachment to ADM vendors
1. Go to https://myaccount.google.com/apppasswords and create new app profile & password associated with your personal flipkart account. Ensure you have 2 step authorization with your flipkart account.
2. The password has to be updated in Step 5 along with your email ID
3. The updated Vendor list along with their email IDs has to be stored in an .xlsx file locally & the path to the file has to be updated in Step 2.
4. The respective vendor files will be processed in a local folder for processing. The folder path has to be updated in Step 4.
5. The file has to be executed in the order mentioned in this file from Step 1 to Step 5.
6. The file contains paths used during testing. Please update before using.

## 1. Install necessary packages in python

In [1]:
# Install packages
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta
import pandas as pd
import seaborn as sns
import requests
import os
from pathlib import Path

#for sending mail
import smtplib
import imghdr
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import glob

C:\Users\prajwal.d1\AppData\Local\Temp\ipykernel_12900\1044736199.py:16: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


## 2. Read Vendor names & their email IDs from local excel file

In [3]:
# Reading a BGV dump database
FwdDump = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Cron_Data_19A.xlsx', dtype=str)

In [4]:
# Reading a database of vendors & email ids
VendorContact = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Vendor_Mail.xlsx', dtype=str)

In [5]:
VendorNames = VendorContact["Vendor Name"].tolist()
VendorMail = VendorContact["Vendor Mail"].tolist()

## 3. Creating file directory & clear old data

In [6]:
# Create a folder and update path here
DownloadPath = "C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Pendency_All"

In [7]:
# Remove all files in the specified folder
os.chdir(f"{DownloadPath}")
all_files = os.listdir()

for f in all_files:
    os.remove(f)

## 4. Processing data for respective vendors

In [8]:
# Create separate files for vendors
attachment_filename_fwd = []

for x in range(len(VendorNames)):
    VendorData_Fwd = FwdDump.loc[FwdDump['Vendor Name'] == VendorNames[x]]
    VendorData_Fwd.to_excel(Path(f"C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Pendency_All\\{VendorNames[x]}_Pendency.xlsx"),index=False)
    attachment_filename_fwd.append(VendorNames[x]+"_Pendency.xlsx")


## 5. Send E-mail to vendor with their respective attachment

In [9]:
# Enter credentials of your account
EMAIL_ADDRESS = 'prajwal.d1@flipkart.com'
EMAIL_PASSWORD = 'fudclnzrtpnsavab'


In [10]:
for y in range(len(VendorNames)):
    msg = EmailMessage()
    msg['Subject'] = "Pendency Data 19th April || -"+VendorNames[y]
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = VendorMail[y]
    msg.set_content("""Dear Partner

   Pendency Data
    
        -> Ensure that all shipments pending for more than 3 days are cleared by EOD.
        -> For shipments pending for over 7 days, kindly provide a Root Cause Analysis (RCA) without fail.
            

      Thanks & Regards,
      Prajwal D|| 7975770738""")
    
    files1 = [attachment_filename_fwd[y]]
    for file in files1:
        with open(file,'rb') as f:
            file_data1 = f.read()
            file_name1 = f.name
        msg.add_attachment(file_data1, maintype='application', subtype='octet-stream', filename=file_name1)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp: 
        smtp.login(EMAIL_ADDRESS,EMAIL_PASSWORD)
        smtp.send_message(msg)    


In [11]:
#FwdDump = pd.read_excel(r'C:\Users\prajwal.d1\Desktop\Pendency\cron_data.xlsx', dtype=str)